In [1]:
import pandas as pd
import numpy as np
import seaborn as sns


import matplotlib.pyplot as plt
import matplotlib as mpl
# from matplotlib import rcParams
# rcParams['font.family'] = 'Malgun Gothic'
plt.rc('font', family='Malgun Gothic')
#plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

import scipy

from datetime import datetime

sns.set(font="AppleGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')


In [2]:
print(mpl.matplotlib_fname())
print(mpl.__file__)

c:\Python38\lib\site-packages\matplotlib\mpl-data\matplotlibrc
c:\Python38\lib\site-packages\matplotlib\__init__.py


In [3]:
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
["NanumGothic" in f]

[False]

In [4]:
naver_df = pd.read_csv('../DAOU/naver.csv', encoding='euc-kr')

In [5]:
naver_df.info()
naver_df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5438 entries, 0 to 5437
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   사업장명                    5438 non-null   object 
 1   업소유형                    852 non-null    object 
 2   업태구분명                   5438 non-null   object 
 3   지번주소                    5427 non-null   object 
 4   도로명주소                   3713 non-null   object 
 5   가까운 지하철역                5438 non-null   object 
 6   지하철역과의 거리               5438 non-null   float64
 7   도보시간                    5438 non-null   float64
 8   위도                      5438 non-null   float64
 9   경도                      5438 non-null   float64
 10  별점                      297 non-null    float64
 11  방문자 리뷰수                 666 non-null    object 
 12  블로그 리뷰수                 496 non-null    object 
 13  한실수                     4371 non-null   float64
 14  양실수                     4347 non-null   

Index(['사업장명', '업소유형', '업태구분명', '지번주소', '도로명주소', '가까운 지하철역', '지하철역과의 거리',
       '도보시간', '위도', '경도', '별점', '방문자 리뷰수', '블로그 리뷰수', '한실수', '양실수', '욕실수',
       '구비시설', '네이버 이런점이 좋아요 총합', '네이버 이런점이 좋아요 {항목:좋아요수}', '데이터갱신일자', '인허가일자',
       '인허가취소일자', '영업상태코드', '폐업일자', '휴업시작일자', '휴업종료일자', '재개업일자', 'naver_pk'],
      dtype='object')

In [6]:
# .. 타입변경
# 방문자, 블로그 리뷰수 실수화
naver_df['방문자 리뷰수'].replace(',','', inplace=True, regex=True)
naver_df['블로그 리뷰수'].replace(',','', inplace=True, regex=True)
naver_df['방문자 리뷰수'] = naver_df['방문자 리뷰수'].astype(float)
naver_df['블로그 리뷰수'] = naver_df['블로그 리뷰수'].astype(float)
naver_df['리뷰수'] = naver_df['방문자 리뷰수'] + naver_df['블로그 리뷰수']

# .. 파생변수 생성
# 객실수 = 한실수 + 양실수
naver_df['객실수'] = naver_df['양실수'] + naver_df['한실수']

# 구비시설개수 = 구비시설 count
naver_df['구비시설'] = naver_df['구비시설'].str.split(",")
naver_df['구비시설 개수'] = naver_df['구비시설'].str.len()

# 운영기간 = 현재일 - 인허가 일자 or 폐업일 - 인허가 일자
naver_df['인허가일자'].replace('-','', inplace=True, regex=True)
naver_df['폐업일자'].replace('-','', inplace=True, regex=True)
naver_df['인허가일자'] = pd.to_datetime(naver_df['인허가일자'])
naver_df['폐업일자'] = pd.to_datetime(naver_df['폐업일자'])
naver_df['운영기간'] = np.nan
cur_date = pd.to_datetime('20230501')
for i in range(naver_df.shape[0]):
    if (pd.isna(naver_df['폐업일자'][i]) == False):
        naver_df['운영기간'][i] = (cur_date - naver_df['인허가일자'][i]).days
    else:
        naver_df['운영기간'][i] = (naver_df['폐업일자'][i] - naver_df['인허가일자'][i]).days

naver_df.info()


C:\Users\tjoeun\AppData\Local\Temp\ipykernel_1352\605929664.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  naver_df['운영기간'][i] = (naver_df['폐업일자'][i] - naver_df['인허가일자'][i]).days
C:\Users\tjoeun\AppData\Local\Temp\ipykernel_1352\605929664.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  naver_df['운영기간'][i] = (cur_date - naver_df['인허가일자'][i]).days


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5438 entries, 0 to 5437
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   사업장명                    5438 non-null   object        
 1   업소유형                    852 non-null    object        
 2   업태구분명                   5438 non-null   object        
 3   지번주소                    5427 non-null   object        
 4   도로명주소                   3713 non-null   object        
 5   가까운 지하철역                5438 non-null   object        
 6   지하철역과의 거리               5438 non-null   float64       
 7   도보시간                    5438 non-null   float64       
 8   위도                      5438 non-null   float64       
 9   경도                      5438 non-null   float64       
 10  별점                      297 non-null    float64       
 11  방문자 리뷰수                 666 non-null    float64       
 12  블로그 리뷰수                 496 non-null    float64 

In [7]:
naver_df.to_csv("../DAOU/naver2.csv", encoding='euc-kr', index=False)

In [8]:
naver_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5438 entries, 0 to 5437
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   사업장명                    5438 non-null   object        
 1   업소유형                    852 non-null    object        
 2   업태구분명                   5438 non-null   object        
 3   지번주소                    5427 non-null   object        
 4   도로명주소                   3713 non-null   object        
 5   가까운 지하철역                5438 non-null   object        
 6   지하철역과의 거리               5438 non-null   float64       
 7   도보시간                    5438 non-null   float64       
 8   위도                      5438 non-null   float64       
 9   경도                      5438 non-null   float64       
 10  별점                      297 non-null    float64       
 11  방문자 리뷰수                 666 non-null    float64       
 12  블로그 리뷰수                 496 non-null    float64 